|<h2>Book:</h2>|<h1><a href="https://open.substack.com/pub/mikexcohen/p/llm-breakdown-16-tokenization-words" target="_blank">50 ML projects to understand LLMs</a></h1>|
|-|:-:|
|<h2>Project:</h2>|<h1><b>[14] Linear semantic axes</b></h1>|
|<h2>Author:<h2>|<h1>Mike X Cohen, <a href="https://sincxpress.com" target="_blank">sincxpress.com</a></h1>|

<br>

<i>Using the code without reading the book may lead to confusion or errors.</i>

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
### matplotlib adjustments (commented lines are for dark mode)

# svg plots (higher-res)
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

plt.rcParams.update({
    # 'figure.facecolor': '#282a2c',
    # 'figure.edgecolor': '#282a2c',
    # 'axes.facecolor':   '#282a2c',
    # 'axes.edgecolor':   '#DDE2F4',
    # 'axes.labelcolor':  '#DDE2F4',
    # 'xtick.color':      '#DDE2F4',
    # 'ytick.color':      '#DDE2F4',
    # 'text.color':       '#DDE2F4',
    'axes.spines.right': False,
    'axes.spines.top':   False,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    'savefig.dpi':300,
})

# **Part 1: Extract and normalize an embeddings matrix**

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

# extract embeddings matrix and convert to numpy
embeddings = model.roberta.embeddings.word_embeddings.weight.detach().numpy()

In [ ]:
# optional normalization
vectorNorms = np.linalg.norm(embeddings,axis=1,keepdims=True)
embeddings_norm = embeddings / vectorNorms
embeddings_norm.shape

In [ ]:
plt.figure(figsize=(12,3))
plt.plot(np.squeeze(vectorNorms),'o',markeredgewidth=.3,markerfacecolor=[.9,.7,.7,.3])
plt.gca().set(xlabel='Token index',ylabel='Embedding norm',
              xlim=[-50,len(vectorNorms)+50])

plt.tight_layout()
plt.savefig('ch3_proj14_part1.png')
plt.show()

In [ ]:
# confirm unit vector
np.linalg.norm(embeddings_norm[300]), np.linalg.norm(embeddings[300])

# **Part 2: Create a "semantic axis"**

In [ ]:
# pick two words to define the axis
word4pos = tokenizer.encode(' future',add_special_tokens=False)
word4neg = tokenizer.encode(' past',add_special_tokens=False)

# confirm they're single-token words
print(word4pos, word4neg)

# but it's best to "de-dimensionalize" them for subsequent plotting
word4pos = word4pos[0]
word4neg = word4neg[0]

In [ ]:
# get the vectors for those words
v2add = embeddings[word4pos]
v2sub = embeddings[word4neg]

# create the "semantic axis" with "raw" vectors
semantic_axis = v2add - v2sub
semantic_axis /= np.linalg.norm(semantic_axis) # post-subtraction normalization

# now starting from the normed vectors
v2add = embeddings_norm[word4pos]
v2sub = embeddings_norm[word4neg]
semantic_axis_from_norm = v2add - v2sub

# print the norms
print(f'Norm of non-normed subtraction: {np.linalg.norm(semantic_axis):.3f}')
print(f'Norm of pre-normed subtraction: {np.linalg.norm(semantic_axis_from_norm):.3f}')

In [ ]:
# visualize
fig = plt.figure(figsize=(12,4))
gs = gridspec.GridSpec(1,3,figure=fig)
ax1 = fig.add_subplot(gs[:-1])
ax2 = fig.add_subplot(gs[-1])


ax1.plot(semantic_axis,'rs',markerfacecolor=[.9,.7,.7,.5],label='Normed after subtraction')
ax1.plot(semantic_axis_from_norm,'bo',markerfacecolor=[.7,.7,.9,.5],label='Subtraction of normed vectors')
ax1.legend()
ax1.set(xlabel='Embeddings dimension',ylabel='Embedding weight',
        xlim=[-10,len(semantic_axis)+10],title='A) Vector differences')

ax2.plot(semantic_axis,semantic_axis_from_norm,'ks',markerfacecolor=[.7,.9,.7,.5])
ax2.set(xlabel='Difference of "raw" vectors',ylabel='Difference of normed vectors',
        title='B) Comparison of difference vectors')

plt.tight_layout()
plt.savefig('ch3_proj14_part2.png')
plt.show()

In [ ]:
print('Norms of the two vectors:')
print(f' {np.linalg.norm(embeddings[word4pos]):.3f}')
print(f' {np.linalg.norm(embeddings[word4neg]):.3f}')

# **Part 3: Project all tokens onto the axis**

In [ ]:
# calculate dot products
dotprods = semantic_axis @ embeddings_norm.T

fig = plt.figure(figsize=(12,3.5))
gs = gridspec.GridSpec(1,3,figure=fig)
ax1 = fig.add_subplot(gs[:-1])
ax2 = fig.add_subplot(gs[-1])

# plot all the similarities
ax1.plot(dotprods,'k.',alpha=.3)
ax1.set(xlabel='Token index',ylabel='Projection',xlim=[-10,len(dotprods)+10],
        title='A) Projections onto semantic axis')

ax2.hist(dotprods,bins='fd',color=[.7,.7,.9],edgecolor='gray')
ax2.set(xlabel='Projection',ylabel='Count',
        xlim=[dotprods.min(),dotprods.max()],yscale='log',
        title='B) Distribution of projection values')

plt.tight_layout()
plt.savefig('ch3_proj14_part3.png')
plt.show()

In [ ]:
# find top and bottom 10 highest scores
top10 = dotprods.argsort()[-10:][::-1]
bot10 = dotprods.argsort()[:10]


# print them out
print('\n10 most positive-projected words:')
print('  Proj.  |   Word')
print('---------+------------')
for widx in top10:
  print(f'  {dotprods[widx]:.3f}  |  "{tokenizer.decode(widx)}"')

print('\n\n10 most negative-projected words:')
print('  Proj.  |   Word')
print('---------+------------')
for widx in bot10:
  print(f' {dotprods[widx]:.3f}  |  "{tokenizer.decode(widx)}"')

# **Part 4: Try other semantic axes**

In [ ]:
# good/evil
# young/old
# big/small

# **Part 5: Try other embeddings**

In [ ]:
# # load BERT tokenizer and model
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
# model = BertModel.from_pretrained('bert-large-uncased')
# embeddings = model.embeddings.word_embeddings.weight.detach().numpy()

In [ ]:
# # GPT2 tokenizer and model
# from transformers import GPT2Tokenizer,GPT2Model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2-large')
# embeddings = model.wte.weight.detach().numpy()